In [7]:
%load_ext autoreload
%autoreload 2

import importlib

import datafunction as dataf
importlib.reload(dataf)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'datafunction' from '/Users/zhangzhilu/Desktop/finalproject_2/temperature_and_ph/code/datafunction.py'>

In [13]:
import pandas as pd

In [10]:
df = dataf.fetch_bacdive_optimum_traits(email = "z1849361659@gmail.com", password = "666002270")

-- Authentication successful --
Successfully logged in.
ERROR: Exacly one parameter is required. Please choose one of the following:
id, taxonomy, sequence, genome, 16s, culturecolno
Data saved to bacdive_optimum_traits.csv


In [3]:
import pandas as pd
df = pd.read_csv("data/customed_data_new_1.csv", encoding='utf-8')

df_opt = df[
    (df["culture_temp.Kind of temperature"] == "optimum") &
    (df["culture_pH.Kind of pH"] == "optimum")
]

df_opt.to_csv("data/filtered_optimum_conditions.csv", index=False)
print("success")

FileNotFoundError: [Errno 2] No such file or directory: 'data/customed_data_new_1.csv'

In [15]:
import pandas as pd

# 读取文件
df = pd.read_csv("data/customed_data_new_1.csv", dtype=str)

# 定义排序权重函数：optimum = 0, growth = 1, 其他 = 2
def kind_sort(value):
    if value == "optimum":
        return 0
    elif value == "growth":
        return 1
    else:
        return 2

# 添加排序权重列（不会保留在最终输出）
df["temp_sort"] = df["culture_temp.Kind of temperature"].apply(kind_sort)
df["ph_sort"] = df["culture_pH.Kind of pH"].apply(kind_sort)

# 排序：先按 strain ID，再按 temp_sort，再按 ph_sort
df_sorted = df.sort_values(
    by=["temp_sort", "ph_sort"]
).drop(columns=["temp_sort", "ph_sort"])  # 删除临时排序列

# 保存结果
df_sorted.to_csv("data/strain_temp_ph_sorted.csv", index=False)
print("✅ 每个菌株的温度和 pH 类型已排序（optimum 在前），结果已保存。")


✅ 每个菌株的温度和 pH 类型已排序（optimum 在前），结果已保存。


In [19]:
import pandas as pd

df = pd.read_csv("data/customed_data_new_1.csv")  # 或 pd.read_csv("your_file.csv")

# 找出空白分隔行（整行全是空）
blank_rows = df.isnull().all(axis=1)

# 对目标列进行 groupby + ffill 填充（只对非空白行做）
df.loc[~blank_rows, 'strains_taxid.tax_level'] = (
    df.loc[~blank_rows]
      .groupby('strains.ID_strains')['strains_taxid.tax_level']
      .transform(lambda col: col.fillna(method='ffill'))
)

# 保存结果
df.to_csv("data/customed_data_new_grouped.csv", index=False)


/var/folders/gl/4n41t2_d5cn43yc6kz0pqhrw0000gn/T/ipykernel_1980/959433160.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .transform(lambda col: col.fillna(method='ffill'))
/var/folders/gl/4n41t2_d5cn43yc6kz0pqhrw0000gn/T/ipykernel_1980/959433160.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .transform(lambda col: col.fillna(method='ffill'))


In [20]:
import pandas as pd

# 1. 读取你的原始 CSV 文件
df = pd.read_csv("data/customed_data_new_1.csv")  # ⬅️ 替换为你的真实文件路径

# 2. 提取包含 "optimum temperature" 的行 + 温度数值
df_temp_opt = df[df["culture_temp.Kind of temperature"] == "optimum"]
df_temp_opt = df_temp_opt[["strains.ID_strains", "culture_temp.Temperature"]]
df_temp_opt = df_temp_opt.rename(columns={"culture_temp.Temperature": "optimum_temperature"})

# 3. 提取包含 "optimum pH" 的行 + pH 数值
df_ph_opt = df[df["culture_pH.Kind of pH"] == "optimum"]
df_ph_opt = df_ph_opt[["strains.ID_strains", "culture_pH.pH"]]
df_ph_opt = df_ph_opt.rename(columns={"culture_pH.pH": "optimum_pH"})

# 4. 合并 temperature 和 pH 信息（根据 strains.ID_）
df_merged = pd.merge(df_temp_opt, df_ph_opt, on="strains.ID_strains", how="outer")

# 5. 提取唯一的菌株信息（例如 species、taxid 等）
meta_cols = ["strains.ID_strains", "strains.Species", "strains_taxid.tax_id", "strains_taxid.tax_level"]
df_meta = df[meta_cols].drop_duplicates(subset="strains.ID_strains")

# 6. 合并菌株元数据 + optimum 条件值
df_final = pd.merge(df_meta, df_merged, on="strains.ID_strains", how="inner")

# 8. 保存结果
df_final.to_csv("optimum_temp_ph_per_strain.csv", index=False)


In [23]:
import pandas as pd
import re

# 读取原始文件（你可以换成自己的文件名）
df = pd.read_csv("data/optimum_data.csv")

df["optimum_pH"] = df["optimum_pH"].astype(str)


# 定义函数：将 “x月x日” 转换成 “MM-DD”
def chinese_date_to_mmdd(val):
    match = re.match(r'(\d{1,2})月(\d{1,2})日', val)
    if match:
        month = int(match.group(1))
        day = int(match.group(2))
        return f"{month:02d}-{day:02d}"
    else:
        return val
# 应用到目标列
df["optimum_pH"] = df["optimum_pH"].apply(chinese_date_to_mmdd)

# 保存修正后的文件
df.to_csv("data/optimum_temp_ph_cleaned_dates.csv", index=False)


In [24]:
import pandas as pd

# 读取原始文件（替换为你自己的文件名）
df = pd.read_csv("data/optimum_data_origin.csv")  # 或 pd.read_excel("your_file.xlsx")

# 删除完全重复的行，只保留第一行
df_unique = df.drop_duplicates()

# 保存结果
df_unique.to_csv("data/optimum_data_origin1.csv", index=False)


In [26]:
import pandas as pd

# 读取 CSV 文件（请替换为你的文件名）
df = pd.read_csv("data/optimum_data_origin1.csv")

# 去除 strain ID 重复的行，只保留第一次出现的
df_unique = df.drop_duplicates(subset=["strains.ID_strains"], keep='first')

# 保存新文件
df_unique.to_csv("data/optimum_data_first.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np

# 读取文件（CSV 或 Excel）
df = pd.read_csv("data/optimum_data_first.csv")  # 或 pd.read_excel("your_file.xlsx")

# 定义一个函数，将范围转换为中间值
def range_to_mean(val):
    try:
        if isinstance(val, str) and '-' in val:
            parts = val.replace('–', '-').split('-')  # 兼容中文破折号
            nums = [float(p.strip()) for p in parts if p.strip()]
            return np.mean(nums)
        else:
            return float(val)
    except:
        return np.nan

# 应用转换函数到两个列
df['optimum_temperature'] = df['optimum_temperature'].apply(range_to_mean)
df['optimum_pH'] = df['optimum_pH'].apply(range_to_mean)

# 保存处理后的结果
df.to_csv("data/optimum_data_mean.csv", index=False)


In [28]:
# only keep species
# 读取 CSV 文件
df = pd.read_csv("data/optimum_data_mean.csv")

# 只保留 tax_level 为 species 的行
df_species = df[df["strains_taxid.tax_level"] == "species"]

# 保存为新文件
df_species.to_csv("data/optimum_data_mean_species.csv", index=False)



In [14]:
# 读取 CSV
df = pd.read_csv("../data/optimum_data_mean_species.csv")

# 提取 taxid 列并转为字符串（确保无 NaN 和 int 型）
taxid_list = df['taxid'].dropna().astype(str).tolist()




In [25]:
dataf.run_taxid2fna(taxid_list)

Running: PATH=/usr/local/bin:$PATH bash /Users/zhangzhilu/Desktop/finalproject_2/temperature_and_ph/code/run_taxid2fna.sh 2486576 2182787 1675854 442870 258515 1720083 2487273 1677857 649746 97477 1120986 1750518 1969806 1615681 946336 2218256 542300 2792603 1765967 2056283 512383 512382 50715 2678518 1232575 1689834 1867749 1382359 1922313 28049 2004485 758826 592050 1789224 28090 2006115 2576829 1096779 449463 2029103 1590614 502577 1136071 478109 478115 478106 478107 478111 502180 478114 478110 252394 1657 519421 390989 227318 1677853 977794 1213861 1813471 504798 547056 585224 547057 1048753 1819600 406431 1427535 2038117 1120938 882440 547461 2293572 2650748 2291546 2094049 559627 559629 887459 1960083 2321394 2057800 1288391 2291215 696000 2044873 502181 1912961 882441 1629072 1734445 1223317 2486853 1547892 533267 446787 1234544 995066 1034798 1873423 304899 1348467 1386098 2072922 2073160 2184697 670289 670288 2086471 1892554 529703 671266 580026 1481665 797535 310575 2527972 2

In [30]:
dataf.run_protein_prediction(
    fasta_dir_path="Users/zhangzhilu/Desktop/finalproject_2/temperature_and_ph/data/labelled_genomes",
    evalue='1e-50',
    nproc=4,
    taxids=None)

[]